# Phase 2: Hardware Characterization

**QubitPulseOpt - Closed-Loop Pulse Optimization for IQM Resonance**

This notebook demonstrates the complete Phase 2 workflow:
1. Hardware backend connection
2. T1 (energy relaxation) characterization
3. T2 (dephasing) characterization
4. Rabi oscillation measurements
5. Randomized Benchmarking (gate fidelity)
6. Configuration updates

**Mode**: Emulator/Simulator (no real hardware required)

**Status**: Ready for headless development

---

## Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Add src to path
sys.path.insert(0, str(Path.cwd().parent.parent / "src"))

# Load environment variables
load_dotenv(Path.cwd().parent.parent / ".env")

print("✓ Setup complete")

## 1. Backend Initialization

Connect to the quantum backend. In headless development mode, we use the emulator.

In [ ]:
from hardware.iqm_backend import IQMBackendManager

# Initialize backend manager
manager = IQMBackendManager()
print(f"Backend manager: {manager}")

# Get emulator backend (use_emulator=True for headless development)
# For real hardware, set use_emulator=False
USE_EMULATOR = True  # Change to False when you have IQM hardware access

backend = manager.get_backend(use_emulator=USE_EMULATOR)
print(f"\n✓ Connected to backend: {backend}")

if hasattr(backend, 'configuration'):
    config = backend.configuration()
    print(f"  Qubits: {config.n_qubits}")
    print(f"  Type: {type(backend).__name__}")

## 2. Initialize Hardware Characterizer

Create the characterizer object that will run all experiments.

In [ ]:
from hardware.characterization import HardwareCharacterizer

# Create characterizer with default settings
characterizer = HardwareCharacterizer(
    backend=backend,
    default_shots=1024,  # Number of measurement shots
    analysis_timeout=300.0  # Analysis timeout in seconds
)

print(f"✓ Characterizer created: {characterizer}")

## 3. T1 Characterization (Energy Relaxation)

Measure how long a qubit stays in the excited state before relaxing to ground state.

In [ ]:
# Run T1 experiment
print("Running T1 experiment...")
t1_result = characterizer.run_t1_experiment(
    qubit=0,
    shots=1024,
    use_emulator=USE_EMULATOR
)

if t1_result['success']:
    print(f"\n✓ T1 Measurement Complete")
    print(f"  T1 = {t1_result['value']*1e6:.2f} ± {t1_result['stderr']*1e6:.2f} μs")
    print(f"  Shots: {t1_result['shots']}")
else:
    print(f"\n✗ T1 experiment failed: {t1_result.get('error', 'Unknown')}")

## 4. T2 Characterization (Dephasing)

Measure quantum coherence time using both Hahn echo and Ramsey methods.

### 4.1 T2 Hahn Echo

Refocuses low-frequency noise using a π pulse.

In [ ]:
# Run T2 Hahn echo
print("Running T2 Hahn echo experiment...")
t2_hahn = characterizer.run_t2_experiment(
    qubit=0,
    shots=1024,
    method='hahn',
    use_emulator=USE_EMULATOR
)

if t2_hahn['success']:
    print(f"\n✓ T2 Hahn Echo Complete")
    print(f"  T2 = {t2_hahn['value']*1e6:.2f} ± {t2_hahn['stderr']*1e6:.2f} μs")
else:
    print(f"\n✗ T2 Hahn failed: {t2_hahn.get('error', 'Unknown')}")

### 4.2 T2* Ramsey

Measures free evolution dephasing (T2*).

In [ ]:
# Run T2 Ramsey
print("Running T2 Ramsey experiment...")
t2_ramsey = characterizer.run_t2_experiment(
    qubit=0,
    shots=1024,
    method='ramsey',
    use_emulator=USE_EMULATOR
)

if t2_ramsey['success']:
    print(f"\n✓ T2 Ramsey Complete")
    print(f"  T2* = {t2_ramsey['value']*1e6:.2f} ± {t2_ramsey['stderr']*1e6:.2f} μs")
else:
    print(f"\n✗ T2 Ramsey failed: {t2_ramsey.get('error', 'Unknown')}")

## 5. Rabi Oscillation Characterization

Measure the relationship between pulse amplitude and rotation angle.

In [ ]:
# Run Rabi experiment
print("Running Rabi experiment...")
rabi_result = characterizer.run_rabi_experiment(
    qubit=0,
    shots=1024,
    use_emulator=USE_EMULATOR
)

if rabi_result['success']:
    print(f"\n✓ Rabi Experiment Complete")
    print(f"  Rabi Frequency = {rabi_result['rate']/1e6:.2f} ± {rabi_result['stderr']/1e6:.2f} MHz")
    print(f"  Rabi Period = {1.0/rabi_result['rate']*1e9:.2f} ns")
else:
    print(f"\n✗ Rabi experiment failed: {rabi_result.get('error', 'Unknown')}")

## 6. Full Characterization Suite

Run all characterization experiments in one call.

In [ ]:
# Run complete characterization
print("Running full characterization suite...")
full_results = characterizer.characterize_qubit(
    qubit=0,
    shots=1024,
    experiments=['T1', 'T2', 'Rabi'],
    use_emulator=USE_EMULATOR
)

if full_results['success']:
    print("\n✓ Full Characterization Complete\n")
    summary = full_results['summary']
    
    print("Summary:")
    if 'T1' in summary:
        print(f"  T1  = {summary['T1']*1e6:.2f} ± {summary.get('T1_stderr', 0)*1e6:.2f} μs")
    if 'T2' in summary:
        print(f"  T2  = {summary['T2']*1e6:.2f} ± {summary.get('T2_stderr', 0)*1e6:.2f} μs")
    if 'rabi_rate' in summary:
        print(f"  Rabi = {summary['rabi_rate']/1e6:.2f} ± {summary.get('rabi_rate_stderr', 0)/1e6:.2f} MHz")
else:
    print("\n✗ Full characterization failed")

## 7. Standard Randomized Benchmarking

Measure average gate fidelity using random Clifford sequences.

In [ ]:
# Run Standard RB
print("Running Standard Randomized Benchmarking...")
rb_result = characterizer.run_randomized_benchmarking(
    qubits=0,
    lengths=[1, 10, 20, 50, 75, 100],  # Clifford sequence lengths
    num_samples=10,  # Number of random sequences per length
    shots=512,
    seed=42,
    use_emulator=USE_EMULATOR
)

if rb_result['success']:
    print("\n✓ Randomized Benchmarking Complete\n")
    print(f"  Error per Clifford (EPC) = {rb_result['epc']:.2e} ± {rb_result['epc_stderr']:.2e}")
    print(f"  Average Gate Fidelity    = {rb_result['fidelity']:.6f}")
    if rb_result['alpha'] is not None:
        print(f"  Depolarizing parameter α = {rb_result['alpha']:.6f} ± {rb_result.get('alpha_stderr', 0):.6f}")
    print(f"\n  Sequence lengths: {rb_result['lengths']}")
    print(f"  Samples per length: {rb_result['num_samples']}")
    print(f"  Total circuits: {len(rb_result['lengths']) * rb_result['num_samples']}")
else:
    print(f"\n✗ RB failed: {rb_result.get('error', 'Unknown')}")

## 8. Interleaved Randomized Benchmarking

Measure the fidelity of a specific gate (e.g., X gate).

In [ ]:
from qiskit.circuit.library import XGate

# Create the gate to test
target_gate = XGate()

# Run Interleaved RB
print("Running Interleaved RB for X gate...")
irb_result = characterizer.run_interleaved_rb(
    qubits=0,
    interleaved_gate=target_gate,
    lengths=[1, 10, 20, 50],
    num_samples=10,
    shots=512,
    seed=42,
    use_emulator=USE_EMULATOR
)

if irb_result['success']:
    print("\n✓ Interleaved RB Complete\n")
    print(f"  Interleaved EPC = {irb_result['epc_interleaved']:.2e} ± {irb_result['epc_interleaved_stderr']:.2e}")
    if irb_result['epc_standard'] is not None:
        print(f"  Standard EPC    = {irb_result['epc_standard']:.2e}")
    if irb_result['gate_error'] is not None:
        print(f"  X Gate Error    = {irb_result['gate_error']:.2e} ± {irb_result.get('gate_error_stderr', 0):.2e}")
        print(f"  X Gate Fidelity = {irb_result['gate_fidelity']:.6f}")
else:
    print(f"\n✗ Interleaved RB failed: {irb_result.get('error', 'Unknown')}")

## 9. Summary of Characterization Results

Compile all results into a summary table.

In [ ]:
import pandas as pd

# Create summary DataFrame
summary_data = []

if t1_result.get('success'):
    summary_data.append({
        'Parameter': 'T1',
        'Value': f"{t1_result['value']*1e6:.2f} μs",
        'Std Error': f"{t1_result['stderr']*1e6:.2f} μs",
        'Status': '✓'
    })

if t2_hahn.get('success'):
    summary_data.append({
        'Parameter': 'T2 (Hahn)',
        'Value': f"{t2_hahn['value']*1e6:.2f} μs",
        'Std Error': f"{t2_hahn['stderr']*1e6:.2f} μs",
        'Status': '✓'
    })

if t2_ramsey.get('success'):
    summary_data.append({
        'Parameter': 'T2* (Ramsey)',
        'Value': f"{t2_ramsey['value']*1e6:.2f} μs",
        'Std Error': f"{t2_ramsey['stderr']*1e6:.2f} μs",
        'Status': '✓'
    })

if rabi_result.get('success'):
    summary_data.append({
        'Parameter': 'Rabi Frequency',
        'Value': f"{rabi_result['rate']/1e6:.2f} MHz",
        'Std Error': f"{rabi_result['stderr']/1e6:.2f} MHz",
        'Status': '✓'
    })

if rb_result.get('success'):
    summary_data.append({
        'Parameter': 'Avg Gate Fidelity',
        'Value': f"{rb_result['fidelity']:.6f}",
        'Std Error': f"{rb_result['epc_stderr']:.2e}",
        'Status': '✓'
    })

if irb_result.get('success') and irb_result.get('gate_fidelity'):
    summary_data.append({
        'Parameter': 'X Gate Fidelity',
        'Value': f"{irb_result['gate_fidelity']:.6f}",
        'Std Error': f"{irb_result.get('gate_error_stderr', 0):.2e}",
        'Status': '✓'
    })

df_summary = pd.DataFrame(summary_data)
print("\nCharacterization Summary:")
print("="*60)
print(df_summary.to_string(index=False))
print("="*60)

## 10. Configuration Update

Update the QubitPulseOpt configuration with measured parameters.

In [ ]:
print("Configuration Update:")
print("="*60)

if full_results.get('success'):
    summary = full_results['summary']
    
    print("\nCharacterized parameters ready for config update:")
    if 'T1' in summary:
        print(f"  T1 = {summary['T1']} s")
    if 'T2' in summary:
        print(f"  T2 = {summary['T2']} s")
    if 'rabi_rate' in summary:
        print(f"  Rabi rate = {summary['rabi_rate']} Hz")
    
    print("\n✓ In production, these would be saved to config/hardware_config.yaml")
    print("✓ Agent would use these values for pulse optimization")
else:
    print("\n⚠️  No characterization results available for config update")

print("="*60)

## Summary

### Phase 2 Complete ✓

This notebook demonstrated:
- ✓ Hardware backend connection (emulator mode)
- ✓ T1, T2, and Rabi characterization
- ✓ Standard and Interleaved Randomized Benchmarking
- ✓ Configuration update workflow

### Next Steps

1. **Phase 3**: Pulse optimization and IQM translation
2. **Phase 4**: Agent orchestration (QubitCalibrator)
3. **Hardware Validation**: When ready, set `USE_EMULATOR=False` and run on real IQM hardware

### Notes

- All experiments run in **emulator mode** - no hardware credits consumed
- Results are simulated but code paths are fully validated
- When switching to real hardware, only change `USE_EMULATOR` flag
- Real hardware characterization will provide actual qubit parameters

---

**Ready for Phase 3!** 🚀